In [11]:
from pandas import read_csv, to_datetime, DataFrame
import matplotlib.pyplot as plt

from scipy.stats import boxcox
import plotly.express as px 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss#,acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from numpy import  random, log,arange, log2, log10
import numpy as np
#from math import log

Visualisation des Données :

Graphique des séries temporelles : Visualisez la série pour identifier les tendances, les saisonnalités, et les anomalies.
Graphique des composantes de la décomposition : Utilisez la décomposition de la série temporelle pour extraire les composantes de tendance, de saisonnalité, et résiduelle.
Statistiques Descriptives :

Résumé statistique : Calculez les statistiques descriptives telles que la moyenne, la médiane, l'écart-type, etc.
Histogramme : Visualisez la distribution des valeurs de la série temporelle.
Test de Stationnarité :

Test de Dickey-Fuller augmenté (ADF) : Vérifiez si la série est stationnaire ou non.
Test de Kwiatkowski-Phillips-Schmidt-Shin (KPSS) : Un autre test pour vérifier la stationnarité.
Analyse de l'Autocorrélation :

Fonction d'Autocorrélation (ACF) : Visualisez les corrélations entre les valeurs de la série à différents décalages.
Fonction d'Autocorrélation partielle (PACF) : Visualisez les corrélations partielles pour mieux comprendre les relations entre les décalages.
Détection et Gestion des Saisonnalités et Tendances :

Décomposition STL : Utilisez la décomposition STL (Seasonal and Trend decomposition using Loess) pour analyser les composantes de la série.
Transformation logarithmique ou différence : Appliquez des transformations pour stabiliser la variance ou rendre la série stationnaire.
Détection des Anomalies et des Points de Rupture :

Détection des valeurs aberrantes : Identifiez les valeurs anormalement élevées ou basses.
Détection des points de rupture : Identifiez les points où la série change de comportement.
Analyse Fréquentielle :

Analyse de Fourier : Identifiez les fréquences dominantes dans la série pour comprendre la saisonnalité.
Cross-Correlation Analysis :

Analyse des corrélations croisées : Si vous avez plusieurs séries temporelles, analysez les relations entre elles.

## Import du jeu de données 

In [12]:
seed = 42
random.seed(seed)
dfTime = read_csv("D:\Alternance_2023_UTT\MS EBDE UTT 23 Prof thesis\Sample_Data\Processed\Product families over time.csv")

df2 = dfTime.set_index('Product Family').T
df2.index = to_datetime(df2.index)

<>:3: SyntaxWarning: invalid escape sequence '\A'
<>:3: SyntaxWarning: invalid escape sequence '\A'
D:\WK\ydongue\AppData\Local\Temp\ipykernel_6016\4091362910.py:3: SyntaxWarning: invalid escape sequence '\A'
  dfTime = read_csv("D:\Alternance_2023_UTT\MS EBDE UTT 23 Prof thesis\Sample_Data\Processed\Product families over time.csv")


## Transformations qu'on va appliquer aux séries pour les rendre stationnaire

In [13]:
import pandas as pd
import numpy as np
from scipy.stats import boxcox
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

def test_stationarity_adf(series):
    # Test ADF
    result = adfuller(series, autolag='AIC')
    p_value = result[1]
    return p_value < 0.05  # Return True if the series is stationary

def determine_petit_truc(series):
    min_positive_value = series[series > 0].min()
    val = 0.00001
    return min_positive_value * val

def differentiate_until_stationary(series, max_diff=3):
    diff_count = 0
    while diff_count < max_diff:
        if test_stationarity_adf(series):
            return series, diff_count
        series = series.diff().dropna()  # Differentiate the series
        diff_count += 1
    return series, diff_count

def process_time_series(series):
    transformations = {
        "loga": 0,
        "Petit_truc": np.nan,
        "ordreP": 0
    }

    original_index = series.index  # Save the original index

    if test_stationarity_adf(series):
        print("The series is already stationary.")
        return series, transformations
    
    # Differentiate the series up to 3 times (initial count not recorded in "Différenciations")
    differentiated_series, initial_diff_count = differentiate_until_stationary(series)

    if test_stationarity_adf(differentiated_series):
        transformations["ordreP"] = initial_diff_count  # Record the initial differentiations
        # Adjust index to match the length of the differentiated series
        differentiated_series.index = original_index[-len(differentiated_series):]
        return differentiated_series, transformations
    
    # Apply the logarithmic transformation
    petit_truc = determine_petit_truc(series)
    transformed_series = np.log(series + petit_truc)
    transformations["loga"] = 1
    transformations["Petit_truc"] = petit_truc
    
    # After the logarithmic transformation, differentiate the series
    differentiated_series, post_log_diff_count = differentiate_until_stationary(transformed_series)
    transformations["ordreP"] = post_log_diff_count  # Record only the differentiations after transformation

    if test_stationarity_adf(differentiated_series):
        # Adjust index to match the length of the differentiated series
        differentiated_series.index = original_index[-len(differentiated_series):]
        return differentiated_series, transformations
    
    # Adjust index to match the length of the differentiated series
    differentiated_series.index = original_index[-len(differentiated_series):]
    return differentiated_series, transformations

# Example usage with fictitious data
if __name__ == "__main__":
    # Initialization of DataFrames to store the results
    df_differentiated = pd.DataFrame()  # For differentiated series
    df_transformations = pd.DataFrame(columns=["family", "loga", "Petit_truc", "ordreP"])  # For applied transformations

    for fam in df2.columns: 
        print("*******")
        print(fam)
        processed_family, transformations = process_time_series(df2[fam])
        df_differentiated[fam] = processed_family  # Save the differentiated series
        transformations["family"] = fam

        # Create a DataFrame from the transformations and remove columns containing only NaNs
        transformations_df = pd.DataFrame([transformations]).dropna(axis=1, how='all')

        # Use pd.concat instead of append
        df_transformations = pd.concat([df_transformations, transformations_df], ignore_index=True)
        print("*******")

    # Display the resulting DataFrames
    print(df_differentiated)
    print(df_transformations)


*******
A
*******
*******
B
The series is already stationary.
*******
*******
C
The series is already stationary.
*******
*******
D
The series is already stationary.
*******
*******
E
The series is already stationary.
*******
*******
F
*******
*******
G
The series is already stationary.
*******
*******
H
*******
*******
I
The series is already stationary.
*******
*******
J
The series is already stationary.
*******
*******
K
*******
*******
L
The series is already stationary.
*******
*******
M
*******
*******
N
*******
*******
O
*******
*******
P
*******
*******
Q
The series is already stationary.
*******
                    A         B         C         D          E          F  \
2020-03-01  -145778.0  119501.0  161600.0  199000.0   536000.0  1173970.0   
2020-04-01    33548.0  113901.0   83800.0  236775.0        0.0 -1278343.0   
2020-05-01   -68616.0   75220.0  143400.0   51750.0  1469000.0  2467319.0   
2020-06-01    47216.0  166397.0  189600.0  188000.0        0.0 -1854166.0   
202

D:\WK\ydongue\AppData\Local\Temp\ipykernel_6016\2787426611.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_transformations = pd.concat([df_transformations, transformations_df], ignore_index=True)


## Transformations pour revenir sur la série originale 
## Ceci pourrait vous être utile si vous voulez vérifier vos transformations

In [14]:

def invert_transformations(df_differentiated, df_transformations, df_original):
    df_reverted = pd.DataFrame(index=df_original.index)

    for index, row in df_transformations.iterrows():
        family = row['family']
        orderP = int(row['ordreP'])
        loga = int(row['loga'])
        petit_truc = row.get('Petit_truc', 0)

        inverted_series = df_differentiated[family].copy()
        revertSerie = pd.Series(index = df_original.index)
        revertSerie.update(df_differentiated[family].copy())
      
        if orderP == 2:
            
            print(family, orderP)
            
            # Retour à l'ordre 1 
            diff_ordre_1 = df_original[family].diff().dropna()
            first_value_ordre_1 = diff_ordre_1.iloc[0]
            vectBuff2 = (first_value_ordre_1 + np.cumsum(inverted_series))
            inverted_series_ordre_1 = np.concatenate(([first_value_ordre_1], vectBuff2))
            
            # Puis retour à l'ordre 0
            
            first_value_orig = df_original[family].iloc[0]
            if loga == 0:
                inverted_series = np.concatenate(([first_value_orig], (first_value_orig + np.cumsum(inverted_series_ordre_1))))
            if loga == 1:
                invbuff = np.concatenate(([first_value_orig], (first_value_orig + np.cumsum(inverted_series_ordre_1))))
                inverted_series = np.exp(invbuff) - petit_truc
            #inverted_series = inverted_series[:len(df_original[family])]

        elif orderP == 1:
            print(family, orderP)
            revertSerie.iloc[1] = df_original[family].iloc[1] - df_original[family].iloc[0]
            
            vectBuff = ( np.cumsum(revertSerie.dropna()) + df_original[family].iloc[0] )
            
            #inverted_series = np.concatenate( ( [df_original[family].iloc[0]] , vectBuff  ) )
            if loga == 0:
                inverted_series = np.concatenate( ( [df_original[family].iloc[0]] , vectBuff  ) )
            if loga == 1:
                invbuff = np.concatenate( ( [df_original[family].iloc[0]] , vectBuff  ) )
                inverted_series = np.exp(invbuff) - petit_truc
            
        elif orderP == 0:    
            inverted_series = df_original[family]

        df_reverted[family] = inverted_series

    return df_reverted


In [15]:
# Sauvegarder les DataFrames dans des fichiers CSV
df_differentiated.to_csv('D:\Alternance_2023_UTT\MS EBDE UTT 23 Prof thesis\Sample_Data\For_Modeling\\series_differenciees.csv')  # Enregistre les séries différenciées
df_transformations.to_csv('D:\Alternance_2023_UTT\MS EBDE UTT 23 Prof thesis\Sample_Data\For_Modeling\\transformations_appliquees.csv', index=False)  # Enregistre les transformations appliquées


<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:3: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:3: SyntaxWarning: invalid escape sequence '\A'
D:\WK\ydongue\AppData\Local\Temp\ipykernel_6016\1442876378.py:2: SyntaxWarning: invalid escape sequence '\A'
  df_differentiated.to_csv('D:\Alternance_2023_UTT\MS EBDE UTT 23 Prof thesis\Sample_Data\For_Modeling\\series_differenciees.csv')  # Enregistre les séries différenciées
D:\WK\ydongue\AppData\Local\Temp\ipykernel_6016\1442876378.py:3: SyntaxWarning: invalid escape sequence '\A'
  df_transformations.to_csv('D:\Alternance_2023_UTT\MS EBDE UTT 23 Prof thesis\Sample_Data\For_Modeling\\transformations_appliquees.csv', index=False)  # Enregistre les transformations appliquées
